In [2]:
import os
import io
import json, glob
import urllib3
from PIL import Image
import PIL.Image
import numpy as np
import pandas as pd
from tenacity import *

In [3]:
!pip install tenacity


In [4]:
def parse_dataset(dataset, outdir, _max=10000):
    """
    parse the dataset to create a list of tuple containing absolute path and url of image
    :param _dataset: dataset to parse
    :param _outdir: output directory where data will be saved
    :param _max: maximum images to download (change to download all dataset)
    :return: list of tuple containing absolute path and url of image
    """
    _fnames_urls = []
    with open(dataset, 'r') as f:
        data = json.load(f)
        for image in data["images"]:
            url = image["url"]
            fname = os.path.join(outdir, "{}.jpg".format(image["imageId"]))
            _fnames_urls.append((fname, url))
    return _fnames_urls[:_max]
def download_image(fnames_and_urls):
    """
    download image and save its with 90% quality as JPG format
    skip image downloading if image already exists at given path
    :param fnames_and_urls: tuple containing absolute path and url of image
    """
    fname, url = fnames_and_urls
    if not os.path.exists(fname):
        http = urllib3.PoolManager(retries=retry(connect=3, read=2, redirect=3))
        response = http.request("GET", url)
        image = Image.open(io.BytesIO(response.data))
        image_rgb = image.convert("RGB")
        image_rgb.save(fname, format='JPEG', quality=90)
        
%cd '/home/ec2-user/SageMaker/imat/'

parse = parse_dataset('train.json', '/home/ec2-user/SageMaker/imat/train_images',100000)

for i in parse:
    download_image(i)

/home/ec2-user/SageMaker/imat


KeyboardInterrupt: 

In [5]:
train_json = json.load(open('/home/ec2-user/SageMaker/imat/train.json'))

In [6]:
num_images = len(train_json['images'])
print(num_images)

1014544


In [13]:
# Get image ids and labels
image_id = []
label_id = []
for i in range(46053):
    image_id.append(int(train_json['annotations'][i]['imageId']))
    label_list = train_json['annotations'][i]['labelId']
    label_str = ''
    for j in range(len(label_list)):
        label_str += ' ' + label_list[j]
    label_str = label_str[1:]
    label_id.append(label_str)

In [14]:
len(label_id)

46053

In [15]:
def get_index(path):
    return int(path.split('/')[-1].split('.')[0])


In [16]:
# Get validation image paths
train_path_format = '/home/ec2-user/SageMaker/imat/train_images/*.jpg'
image_path = sorted(glob.glob(train_path_format), key= get_index)

In [17]:
len(image_path)

46053

In [18]:
train_csv = pd.DataFrame({'imageId': image_id, 'labelId': label_id, 'imagePath': image_path})

In [19]:
train_csv.head()

,imageId,labelId,imagePath
0,1,95 66 137 70 20,/home/ec2-user/SageMaker/imat/train_images/1.jpg
1,2,36 66 44 214 105 133,/home/ec2-user/SageMaker/imat/train_images/2.jpg
2,3,170 66 97 153 105 138,/home/ec2-user/SageMaker/imat/train_images/3.jpg
3,4,18 66 44 153 164,/home/ec2-user/SageMaker/imat/train_images/4.jpg
4,5,189 66 176 137 153 74 181 171 138,/home/ec2-user/SageMaker/imat/train_images/5.jpg


In [20]:
train_csv.shape

(46053, 3)

In [21]:
train_csv.to_csv('train.csv', index=False)